## Chase Transaction Load

In [41]:
import os
os.getcwd() 

import pandas as pd
# import matplotlib.pyplot as plt

In [34]:

 # Read the data file into a DataFrame
file_path = 'sources/Chase8461_Activity20231118.CSV'

c_data = pd.read_csv(file_path)

# Replace spaces with underscores in column names
c_data.columns = c_data.columns.str.replace(' ', '_')

# Convert all text data in the DataFrame to lowercase (done to help w matching rules)
text_columns = ['Description','Category','Type','Memo']
c_data[text_columns] = c_data[text_columns].apply(lambda x: x.astype(str).str.lower())

# Convert 'Date' column to datetime type with explicit format
c_data['Transaction_Date'] = pd.to_datetime(c_data['Transaction_Date'], format='%m/%d/%Y')
c_data['Post_Date'] = pd.to_datetime(c_data['Post_Date'], format='%m/%d/%Y')

# # Extract year and month from the 'Date' column
# c_data['Year'] = c_data['Date'].dt.year
# c_data['Month'] = c_data['Date'].dt.month


# Verify the updated data type of the 'Date' column
print("Data type of 'Date' column after conversion:", c_data['Transaction_Date'].dtype)
print("Data type of 'Date' column after conversion:", c_data['Post_Date'].dtype)

# Add outliers column and initialize it as null
c_data['outliers'] = 0

# Add NNWS (Needs/Wants/Savings) column and initialize it as null. First broad bucket to group transactions
c_data['NWS'] = 'NULL'
# Add Label2 to capture more granular categories
c_data['Label2'] = 'NULL'

# Strip whitespace from all string columns
c_data = c_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# # Perform a string replace operation on the "ColumnName" column
# c_data['Original_Description'] = c_data['Original_Description'].str.replace('pos purch - ', ' ')
# c_data['Description'] = c_data['Original_Description'].str.replace('pos purch - ', ' ')

# # Remove commas from the some text columns to help avoid loading issues in looker (if used)
# c_data['Notes'] = c_data['Notes'].str.replace(',', '')
# c_data['Description'] = c_data['Description'].str.replace(',', '')
# c_data['Original_Description'] = c_data['Original_Description'].str.replace(',', '')

c_data.head()

Data type of 'Date' column after conversion: datetime64[ns]
Data type of 'Date' column after conversion: datetime64[ns]


,Transaction_Date,Post_Date,Description,Category,Type,Amount,Memo,outliers,NWS,Label2
0,2023-11-14,2023-11-16,baja burrito,food & drink,sale,-21.70,nan,0,NULL,NULL
1,2023-11-14,2023-11-15,amazon.com*c11dk1ei3,shopping,sale,-16.72,nan,0,NULL,NULL
2,2023-11-14,2023-11-15,amzn mktp us*wp8d95sg3,shopping,sale,-20.75,nan,0,NULL,NULL
3,2023-11-13,2023-11-13,axs.comopry ryman tix,entertainment,sale,-162.50,nan,0,NULL,NULL
4,2023-11-11,2023-11-13,trader joe s #739,groceries,sale,-127.77,nan,0,NULL,NULL


In [35]:
print(c_data.dtypes)

Transaction_Date    datetime64[ns]
Post_Date           datetime64[ns]
Description                 object
Category                    object
Type                        object
Amount                     float64
Memo                        object
outliers                     int64
NWS                         object
Label2                      object
dtype: object


In [36]:
# Define the categorize_categories function
def categorize_categories(df):
    # Create a dictionary to map original categories to new values
    category_mapping = {
        'food': 'food',
        'food & drink': 'food',
        'restaurants': 'food',
        'cafe': 'food',
        'coffee': 'food',
        'coffee shops': 'food',
        'pharmacy': 'food',
        'groceries': 'food',
        'auto': 'auto',
        'gas & fuel': 'auto',
        'food & dining': 'auto',
        'ride share': 'auto',
        'parking': 'auto',
        'rental car & taxi': 'auto',
        'transportation': 'auto',
        'hotel': 'discretionary',
        'venmo': 'discretionary',
        'travel': 'discretionary',
        'entertainment': 'discretionary',
        'amusement': 'discretionary',
        'clothing': 'discretionary',
        'dating': 'discretionary',
        'transfer': 'transfer',
        'concert': 'discretionary',
        'club': 'discretionary',
        'movie': 'discretionary',
        'Dentist': 'utilities',
        'Doctor': 'utilities',
        'Internet': 'utilities',
        'bills': 'utilities',
        'utilities': 'utilities',
        'Life Insurance': 'utilities',
        'bills & utilities': 'utilities',
        'Mobile Phone': 'utilities',
        'business services': 'utilities',
        'Loans': 'loans',
        'alcohol & bars': 'discretionary',
        'gift': 'discretionary',
        'gym': 'investment',
        'auto & transport': 'auto',
        'music': 'discretionary',
        'fast food': 'food',
        'sporting goods': 'discretionary',
        'charity': 'discretionary',
        'books': 'investment',
        'electronics & software': 'discretionary',
        'shopping': 'discretionary',
        'investments': 'investment',
        'paycheck': 'income',
        'personal care': 'discretionary',
        'misc expenses': 'discretionary',
        'fees & charges': 'bills',
        'mortgage & rent': 'bills',
        'venmo payment': 'bills',
        'advertising': 'discretionary',
        'podcast': 'entertainment',
        'arts': 'discretionary',
        'air travel': 'travel',
        'kids': 'discretionary',
        'newspapers & magazines': 'entertainment',
        'federal tax': 'bills',
        'home improvement': 'utilities',
        'books & supplies': 'investment',
        'uncategorized': 'uncategorized',
        'movies & dvds': 'entertainment',
        'health & fitness': 'investment',
        'income': 'discretionary',
        'doctor': 'health',
        'cash & atm': 'bills',
        'office supplies': 'discretionary',
        'spa & massage': 'discretionary',
        'laundry': 'discretionary',
        'auto insurance': 'auto',
        'hair': 'discretionary',
        'shipping': 'discretionary',
        'service & parts': 'auto',
        'bank fee': 'bills',
        'home services': 'utilities',
        'finance charge': 'bills',
        'atm fee': 'bills',
        'life insurance': 'bills',
        'dentist': 'health',
        'public transportation': 'auto',
        'furnishings': 'discretionary',
        'mobile phone': 'bills',
        'home': 'utilities',
        'sports': 'discretionary',
        'gifts & donation': 'discretionary',
        'bitcoin investment': 'investment',
        'gifts & donations': 'discretionary',
        'television': 'entertainment',
        'vacation': 'travel',
        'therapy': 'health',
        'lawn & garden': 'utilities',
        'classes': 'investment',
        'hca cafe': 'food',
        'credit card payments': 'bills',
        'printing': 'discretionary',
        'credit card payment': 'bills',
        'home supplies': 'utilities',
        'kids activities': 'discretionary',
        'education': 'investment',
        'hobbies': 'discretionary',
        'question?': 'uncategorized',
        'financial': 'investment',
        'home phone': 'utilities',
        'auto payment	': 'auto',
        'returned purchase': 'discretionary',
        'tax advisor': 'bills',
        'internet': 'bills',
        'attorney fee': 'bills',
        'legal': 'bills',
        'return': 'discretionary',
        'gas': 'auto', 
        'automotive': 'auto', 
        'pets': 'discretionary',
        'golf': 'discretionary',
        'comedy club': 'discretionary',
        'cpa fees': 'bills',
        'buy': 'loans',
        'auto loan': 'loans',
        'endurance race': 'discretionary',
        'late fee': 'bills',
        'toys': 'discretionary',
        'transfer for cash spending': 'discretionary',
        'hair': 'discretionary',
        'tuition': 'discretionary',
    }

    # Create a new column 'Cat2' based on the mapping
    df['Cat2'] = df['Category'].str.lower().map(category_mapping)

    return df

# Call the categorize_categories function with your 'c_data' DataFrame
c_data_recategorized = categorize_categories(c_data)


In [37]:
# Custom function to apply recategorization logic
def recategorize_transactions(row):
    if 'climb' in row['Description'] and row['Amount'] > 80:
        return 'gym'
    elif 'hos corp' in row['Description'] and row['Amount'] < 25:
        return 'restaurants'    
    elif 'new loan' in row['Description'] and row['Amount'] > 10000:
        return 'auto loan'    
    elif 'hbo now' in row['Description']:
        return 'entertainment'
    elif 'robinhood' in row['Description']:
        return 'investments'    
    elif 'dividend' in row['Description']:
        return 'investments'   
    elif 'coinbase' in row['Description']:
        return 'investments'  
    elif 'airbnb' in row['Description']:
        return 'hotel'
    elif 'walgreens' in row['Description']:
        return 'shopping'
    elif 'acme feed' in row['Description']:
        return 'restaurants'
    elif 'ach trans - select rwds pymt' in row['Description'] and row['Amount'] > 2792:
        return 'travel'
    else:
        return row['Category']

# Apply the recategorization logic to the DataFrame
c_data_recategorized['Category'] = c_data_recategorized.apply(recategorize_transactions, axis=1)

In [38]:
def unique_categories(dataframe):
    # Get the distinct category values into a list
    return dataframe['Category'].unique().tolist()

distinct_categories = unique_categories(c_data)
# Print the list of distinct categories
# print(distinct_categories)

In [39]:
# Custom function to apply subscription labels based on description content
def label_subscriptions(row):
    if 'lucid' in row['Description']:
        return 'subscription - productivity'
    elif 'hbo' in row['Description'] or 'patreon' in row['Description'] or 'samharris' in row['Description'] or 'spotify' in row['Description']:
        return 'subscription - media'
    elif 'amazon prime' in row['Description']:
        return 'subscription - shopping'
    elif 'airbnb' in row['Description']:
        return 'travel hotel'
    elif 'fahrenheit yoga' in row['Description'] or 'climb' in row['Description']:
        return 'subscription - fitness'
    elif 'ach trans - select rwds pymt' in row['Description'] and row['Amount'] > 2792:
        return 'vegas vacation airbnb'
    else:
        return row['Label2']

# Apply the recategorization logic to the DataFrame
c_data_recategorized['Label2'] = c_data_recategorized.apply(label_subscriptions, axis=1)


In [40]:
c_data

,Transaction_Date,Post_Date,Description,Category,Type,Amount,Memo,outliers,NWS,Label2,Cat2
0,2023-11-14,2023-11-16,baja burrito,food & drink,sale,-21.70,nan,0,NULL,NULL,food
1,2023-11-14,2023-11-15,amazon.com*c11dk1ei3,shopping,sale,-16.72,nan,0,NULL,NULL,discretionary
2,2023-11-14,2023-11-15,amzn mktp us*wp8d95sg3,shopping,sale,-20.75,nan,0,NULL,NULL,discretionary
3,2023-11-13,2023-11-13,axs.comopry ryman tix,entertainment,sale,-162.50,nan,0,NULL,NULL,discretionary
4,2023-11-11,2023-11-13,trader joe s #739,groceries,sale,-127.77,nan,0,NULL,NULL,food
5,2023-11-12,2023-11-13,nespresso usa inc,groceries,sale,-133.88,nan,0,NULL,NULL,food
6,2023-11-11,2023-11-12,nytimes*nytimes disc,bills & utilities,sale,-4.00,nan,0,NULL,NULL,utilities
7,2023-11-10,2023-11-12,the amsterdam local co,food & drink,sale,-10.09,nan,0,NULL,NULL,food
8,2023-11-11,2023-11-12,apple.com/bill,shopping,sale,-2.48,nan,0,NULL,NULL,discretionary
9,2023-11-11,2023-11-12,amazon.com*qn5z60ar3,shopping,sale,-19.65,nan,0,NULL,NULL,discretionary
